# Assignment 1 - Part of Speech Tagging

## Dependencies

In [60]:
# !pip install lightning
# !pip install torchtext.data
# !pip install torchtext
# !pip install torch

In [61]:
# TODO: remove unused dependencies

# file management
import sys
import shutil
import urllib
import tarfile
from pathlib import Path
import zipfile
import os

# DL framework
from torchmetrics import Accuracy, F1Score
from lightning.pytorch.loggers import TensorBoardLogger


# dataframe management
import pandas as pd

# data manipulation
import numpy as np

# for readability
from typing import Iterable
from tqdm import tqdm

## TASK 1: Corpus

### Instructions

* **Download** the corpus.
* **Encode** the corpus into a pandas.DataFrame object.
* **Split** it in training, validation, and test sets.

#### Download the corpus

In [62]:
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)
        
def download_url(download_path: Path, url: str):
    with DownloadProgressBar(unit='B', unit_scale=True,
                             miniters=1, desc=url.split('/')[-1]) as t:
        urllib.request.urlretrieve(url, filename=download_path, reporthook=t.update_to)

        
def download_dataset(download_path: Path, url: str):
    print("Downloading dataset...")
    download_url(url=url, download_path=download_path)
    print("Download complete!")

def extract_dataset(download_path: Path, extract_path: Path):
    print("Extracting dataset... (it may take a while...)")
    with zipfile.ZipFile(download_path, 'r') as zip_file:
        zip_file.extractall(extract_path)

    print("Extraction completed!")

    Path.unlink(download_path)
    print("Deleted .zip dataset file")

In [63]:
url = "https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/packages/corpora/dependency_treebank.zip"
dataset_name = "dependency_treebank"

print(f"Current work directory: {Path.cwd()}")

dataset_folder = Path.cwd().joinpath("Datasets")

if not dataset_folder.exists():
    dataset_folder.mkdir(parents=True)

dataset_zip_path = dataset_folder.joinpath("dependency_treebank.zip")
dataset_path = dataset_folder.joinpath(dataset_name)

if not dataset_zip_path.exists():
    download_dataset(dataset_zip_path, url)

if not dataset_path.exists():
    extract_dataset(dataset_zip_path, dataset_folder)
  

Current work directory: c:\Users\merli\OneDrive\Documenti\University\Anno 5\Natural Language Processing\NLP\A1


#### Encode the corpus into a pandas.DataFrame object and split it into train, validation and test sets

The corpus contains 200 documents.

   * **Train**: Documents 1-100
   * **Validation**: Documents 101-150
   * **Test**: Documents 151-199

In [64]:
dataframe_rows = []  # list for DataFrame rows
id = 0

for i, file_path in enumerate(sorted(dataset_path.iterdir())):
    if file_path.is_file(): # split corpus documents in the tree categories: train, validation, tests
        if 1 <= i + 1 <= 100:
            split = 'train'
        elif 101 <= i + 1 <= 150:
            split = 'validation'
        else:
            split = 'test'

        with file_path.open(mode='r', encoding='utf-8') as text_file: # read corpus lines
            lines = text_file.readlines()
                
        for line in lines:
            fields = line.strip().split('\t')
            if len(fields) == 1:
                id = id + 1
            if len(fields) >= 2:
                text = fields[0]  # store the first field as 'text'
                POS = fields[1]   # store the second field as 'POS'
                dataframe_row = {  #build DataFrame rows
                    "text": text,
                    "POS": POS,
                    "split": split,
                    "id": id
                }

                dataframe_rows.append(dataframe_row) #append rows
# corpus DataFrame
corpus_df = pd.DataFrame(dataframe_rows) 

#### Data inspection

In [65]:
corpus_df.head(10)

,text,POS,split,id
0,Pierre,NNP,train,0
1,Vinken,NNP,train,0
2,",",",",train,0
3,61,CD,train,0
4,years,NNS,train,0
5,old,JJ,train,0
6,",",",",train,0
7,will,MD,train,0
8,join,VB,train,0
9,the,DT,train,0


In [66]:
# Train, test, validation split
df_train = corpus_df[corpus_df['split'] == 'train'].drop(columns=['split'])
df_test = corpus_df[corpus_df['split'] == 'test'].drop(columns=['split'])
df_val = corpus_df[corpus_df['split'] == 'validation'].drop(columns=['split'])

In [67]:
print("Dataframe structure:")
print(corpus_df)
print()

print("Total rows %d" % (len(corpus_df)))
print()

Dataframe structure:
          text  POS  split    id
0       Pierre  NNP  train     0
1       Vinken  NNP  train     0
2            ,    ,  train     0
3           61   CD  train     0
4        years  NNS  train     0
...        ...  ...    ...   ...
94079  quarter   NN   test  3715
94080       of   IN   test  3715
94081     next   JJ   test  3715
94082     year   NN   test  3715
94083        .    .   test  3715

[94084 rows x 4 columns]

Total rows 94084



## TASK 2: Text encoding

### Instructions

* Embed words using **GloVe embeddings**.
* You are **free** to pick any embedding dimension.
* [Optional] You are free to experiment with text pre-processing: **make sure you do not delete any token!**

### Embed words unsing GloVe embeddings

Encode text into numerical format

In [68]:
from torchtext.vocab import GloVe, build_vocab_from_iterator

import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import pytorch_lightning as pl
from lightning.pytorch import loggers as pl_logger

In [69]:
def load_embedding_model(embedding_dimension: int = 300):
    emb_model = GloVe(name="6B", dim=embedding_dimension)
    return emb_model

In [70]:
iterator = ([text] for text in corpus_df["POS"].unique())
vocab = build_vocab_from_iterator(iterator)


class CorpusDataset(Dataset):
    def __init__(self, dataframe: pd.DataFrame, embedder):
        min_id = dataframe['id'].min()
        dataframe['id'] = dataframe['id'] - min_id
        self.dataframe = dataframe.groupby("id")
        self.embedder = embedder

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        sentence = self.dataframe.get_group(idx)
        text = sentence['text'].to_list()
        POS = sentence['POS'].to_list()
        
        POS = torch.Tensor([vocab[token] for token in POS])
        
        POS_one_hot = torch.nn.functional.one_hot(POS.to(torch.int64), num_classes=len(vocab))
        embedded_text = self.embedder.get_vecs_by_tokens(text)
        
        return embedded_text, POS_one_hot


In [71]:
# Definition of the dataset
EMBEDDING_DIM = 50
embedder = load_embedding_model(EMBEDDING_DIM)
dataset_train = CorpusDataset(df_train, embedder)
dataset_test = CorpusDataset(df_test, embedder)
dataset_val = CorpusDataset(df_val, embedder)


# TODO - test if it works in the LSTM training
def my_collate(batch):
    sequences, labels = zip(*batch)
    sequences_padded = torch.nn.utils.rnn.pad_sequence(sequences, batch_first=True, padding_value=0)
    labels_padded = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=0)
    
    
    sequences_padded = sequences_padded.type(torch.float)
    labels_padded = labels_padded.type(torch.float)
    
    
    return [sequences_padded, labels_padded]

train_loader = DataLoader(dataset_train, batch_size=3, collate_fn=my_collate)
val_loader = DataLoader(dataset_val, batch_size=3, collate_fn=my_collate)
test_loader = DataLoader(dataset_test, batch_size=3, collate_fn=my_collate)

In [76]:
class BiLSTMModel(pl.LightningModule):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(BiLSTMModel, self).__init__()
        self.output_dim = output_dim
        self.lstm = nn.LSTM(input_size=input_dim, 
                            hidden_size=hidden_dim, 
                            num_layers=num_layers, 
                            batch_first=True, 
                            bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)  # Multiplied by 2 due to the bidirectionality

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        # lstm_out (batch_size, seq_length, hidden_size * 2)
        out = self.fc(lstm_out)
        # out (batch_size, seq_length, output_dim)
        return out

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        # Change shape to be (batchsize, classes, sequence_len) to compute loss function
        # TODO: Edo: should we normalize the loss by sequence_len?? Otherwise longer phrases influence the loss more
        y = torch.movedim(y, 1, 2)
        y_hat = torch.movedim(y_hat, 1, 2)
        loss = nn.functional.cross_entropy(y_hat, y)

        # TODO - check if it is a legit way to compute F1
        y_class = torch.zeros(y.shape[0], y.shape[1]) 
        y_class = torch.argmax(y, dim=2)
        y_class = torch.flatten(y)

        y_hat_class = torch.zeros(y_hat.shape[0], y.shape[1]) 
        y_hat_class = torch.argmax(y_hat, dim=2)
        y_hat_class = torch.flatten(y_hat)

        f1 = F1Score(task="multiclass", num_classes=self.output_dim)
        f1_score = f1(y_hat_class, y_class)

        self.log_dict({'train_f1': f1_score, 'step': self.current_epoch}, on_epoch=True, prog_bar=True, logger=True, on_step=False)
        self.log_dict({'train_loss': loss, 'step': self.current_epoch}, on_epoch=True, prog_bar=True, logger=True, on_step=False)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        y = torch.movedim(y, 1, 2)
        y_hat = torch.movedim(y_hat, 1, 2)
        loss = nn.functional.cross_entropy(y_hat, y)
        self.log_dict({'val_loss': loss, 'step': self.current_epoch}, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        y = torch.movedim(y, 1, 2)
        y_hat = torch.movedim(y_hat, 1, 2)
        loss = nn.functional.cross_entropy(y_hat, y)

        self.log_dict({'test_loss': loss, 'step': self.current_epoch}, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer


In [73]:
version=1

In [74]:
output_dim = len(df_train["POS"].unique())
input_dim = EMBEDDING_DIM
hidden_dim = 28
max_epochs = 10

save_dir = "logs"
load_model = False
train_model = True 
model_ckpt = "baseline.ckpt"


PATH = os.path.join(save_dir, "lightning_logs", "version_" + str(version), "checkpoints", model_ckpt)


if load_model:
    model = BiLSTMModel.load_from_checkpoint(PATH, input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=1)
else:
    model = BiLSTMModel(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=1)

if train_model:
    tb_logger = TensorBoardLogger(version=version, save_dir='logs')
    trainer = pl.Trainer(max_epochs=max_epochs, logger=tb_logger, default_root_dir='./', log_every_n_steps=1)
    trainer.fit(model, train_loader, val_loader)
    trainer.save_checkpoint(PATH)
    version += 1


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
d:\Users\edo\envs\nlp\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:630: Checkpoint directory logs\lightning_logs\version_2\checkpoints exists and is not empty.

  | Name | Type   | Params
--------------------------------
0 | lstm | LSTM   | 17.9 K
1 | fc   | Linear | 2.6 K 
--------------------------------
20.5 K    Trainable params
0         Non-trainable params
20.5 K    Total params
0.082     Total estimated model params size (MB)


d:\Users\edo\envs\nlp\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
d:\Users\edo\envs\nlp\Lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\result.py:211: You called `self.log('step', ...)` in your `validation_step` but the value needs to be floating point. Converting it to torch.float32.
d:\Users\edo\envs\nlp\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Epoch 9: 100%|██████████| 622/622 [00:25<00:00, 23.99it/s, v_num=2, val_loss=0.405, step=9.000, train_f1=0.000, train_loss=0.337]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 622/622 [00:25<00:00, 23.97it/s, v_num=2, val_loss=0.405, step=9.000, train_f1=0.000, train_loss=0.337]


In [77]:
# test the model
trainer.test(model, dataloaders=test_loader)

d:\Users\edo\envs\nlp\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Testing DataLoader 0:   0%|          | 0/202 [00:41<?, ?it/s]


TypeError: LightningModule.log() missing 1 required positional argument: 'value'

In [78]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 15512), started 0:33:54 ago. (Use '!kill 15512' to kill it.)

In [ ]:
'''
class Model1(pl.LightningModule):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(Model1, self).__init__()
        self.output_dim = output_dim
        self.lstm1 = nn.LSTM(input_size=input_dim, 
                            hidden_size=hidden_dim, 
                            num_layers=num_layers, 
                            batch_first=True, 
                            bidirectional=True)
        
       self.fc = nn.Linear(hidden_dim * 2, output_dim)  # Multiplied by 2 due to bidirectionality
        
        # Additional LSTM layer
        self.lstm2 = nn.LSTM(input_size=hidden_dim * 2, 
                            hidden_size=hidden_dim, 
                            num_layers=num_layers, 
                            batch_first=True, 
                            bidirectional=True)



    def forward(self, x):
        lstm_out1, _ = self.lstm1(x)
        lstm_out2, _ = self.lstm2(lstm_out1)
        out = self.fc(lstm_out2)
        return out

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.functional.cross_entropy(y_hat, y)

        y_class = torch.argmax(y, dim=2).flatten()
        y_hat_class = torch.argmax(y_hat, dim=2).flatten()
        
        f1 = F1Score(task="multiclass", num_classes=self.output_dim).to(device)
        f1_score = f1(y_hat_class, y_class)

        self.log('train_f1', f1_score, on_epoch=True, prog_bar=True, logger=True, on_step=False)
        self.log('train_loss', loss, on_epoch=True, prog_bar=True, logger=True, on_step=False)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.functional.cross_entropy(y_hat, y)
        
        y_class = torch.argmax(y, dim=2).flatten()
        y_hat_class = torch.argmax(y_hat, dim=2).flatten()
        
        self.log('val_loss', loss, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.functional.cross_entropy(y_hat, y)
        
        y_class = torch.argmax(y, dim=2).flatten()
        y_hat_class = torch.argmax(y_hat, dim=2).flatten()
        
        self.log('test_loss', loss, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
'''

In [ ]:
'''
model1 = Model1(input_dim=input_dim, hidden_dim=128, output_dim=output_dim, num_layers=1)
trainer = pl.Trainer(max_epochs=10)
trainer.fit(model1, train_loader, val_loader)
'''

In [ ]:
'''
trainer.test(model1, dataloaders=test_loader)
'''